In [42]:
import pandas as pd 
from collections import defaultdict


In [ ]:
entity_df = pd.read_csv("entity_data.csv")

name_to_id_map = defaultdict(list)
for name, eid in zip(
    entity_df['entity_name'].str.lower().str.replace(" ", "", regex=False),
    entity_df['id']
):
    name_to_id_map[name].append(eid)

id_to_name_map = dict(
    zip(entity_df['id'], entity_df['entity_name'].str.lower().str.strip())
)

entity_to_id_map = defaultdict(list)
for entity, eid in zip(entity_df['entity'], entity_df['id']):
    entity_to_id_map[entity].append(eid)

entity_to_name_map = dict(
    zip(entity_df['entity'], entity_df['entity_name'].str.lower().str.strip())
)

In [44]:
def compute_metrics_tail(row):
    preds = [p.strip() for p in row["Prediction"].split("|")]
    answer = row["Answer"]
    
    if answer in preds:
        rank = preds.index(answer)
        hist1 = 1 if rank == 0 else 0
        hist3 = 1 if rank < 3 else 0
        hist10 = 1 if rank < 10 else 0
        mrr = 1 / (rank + 1)
    else:
        hist1, hist3, hist10, mrr = 0, 0, 0, 0.0
    
    return pd.Series([hist1, hist3, hist10, mrr])

def compute_metrics_head(row):
    preds = [p.strip() for p in row["Prediction"].split("|")]
    answer = row["HeadEntity"]
    
    if answer in preds:
        rank = preds.index(answer)
        hist1 = 1 if rank == 0 else 0
        hist3 = 1 if rank < 3 else 0
        hist10 = 1 if rank < 10 else 0
        mrr = 1 / (rank + 1)
    else:
        hist1, hist3, hist10, mrr = 0, 0, 0, 0.0
    
    return pd.Series([hist1, hist3, hist10, mrr])

In [ ]:
output_tail_df = pd.read_json("output_tail.txt", lines=True)
output_tail = output_tail_df.head(1005).copy()
output_tail["Answer"] = output_tail["Answer"].map(entity_to_name_map.get)

output_tail[["hist1", "hist3", "hist10", "mrr"]] = output_tail.apply(compute_metrics_tail, axis=1)


n = len(output_tail)

tail_scores = {
    "hit@1": output_tail["hist1"].sum() / n,
    "hit@3": output_tail["hist3"].sum() / n,
    "hit@10": output_tail["hist10"].sum() / n,
    "MRR": output_tail["mrr"].mean()
}

tail_scores = pd.DataFrame([tail_scores])
tail_scores

,hit@1,hit@3,hit@10,MRR
0,0.296,0.403,0.489,0.352121


In [ ]:
output_head_df = pd.read_json("output_head.txt", lines=True)
output_head = output_head_df.head(1005).copy()
output_head["HeadEntity"] = output_tail["HeadEntity"].map(entity_to_name_map.get)

output_head[["hist1", "hist3", "hist10", "mrr"]] = output_head.apply(compute_metrics_head, axis=1)


n = len(output_head)

head_scores = {
    "hit@1": output_head["hist1"].sum() / n,
    "hit@3": output_head["hist3"].sum() / n,
    "hit@10": output_head["hist10"].sum() / n,
    "MRR": output_head["mrr"].mean()
}

head_scores = pd.DataFrame([head_scores])
head_scores


,hit@1,hit@3,hit@10,MRR
0,0.238,0.38,0.453,0.308892


In [ ]:
average_score = pd.concat([head_scores, tail_scores]).mean().to_frame()
average_score

,0
hit@1,0.267000
hit@3,0.391500
hit@10,0.471000
MRR,0.330507
